<a href="https://colab.research.google.com/github/vanshika-0305/Music-Generator/blob/main/streamlit(Music_gen).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload the file


Saving music_model_weights.weights.h5 to music_model_weights.weights.h5


In [2]:
!mkdir -p /content/project_folder
!mv music_model_weights.weights.h5 /content/project_folder/


In [3]:
!pip install streamlit tensorflow numpy pretty_midi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.5 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=8ce5f9a14c78c29c41376e27c919513b327a094d8005256ae231e1cadd9d5c74
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [4]:
!pip install pyngrok


In [14]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 3s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [5]:
code="""import streamlit as st
import pretty_midi
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import os

# Load the trained model
@st.cache_resource
def load_model():
    model = Sequential([
        LSTM(128, input_shape=(50, 1), return_sequences=True),
        Dropout(0.3),
        LSTM(128, return_sequences=True),
        Dropout(0.3),
        LSTM(128),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(vocab), activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.load_weights("music_model_weights.h5")  # Load your model weights
    return model

# Generate music
def generate_music(model, int_to_note, start_sequence, num_notes=100, temperature=1.0):
    generated = []
    pattern = start_sequence

    for _ in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1)) / float(len(vocab))
        predictions = model.predict(prediction_input, verbose=0)[0]

        # Apply temperature scaling for randomness
        predictions = np.log(predictions + 1e-10) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        index = np.random.choice(len(probabilities), p=probabilities)

        result = int_to_note[index]
        generated.append(result)

        pattern.append(index)
        pattern = pattern[1:]

    return generated

# Convert notes to MIDI
def notes_to_midi(notes, output_file="output.mid"):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Piano

    start = 0
    for note in notes:
        note_number = int(note)
        note_obj = pretty_midi.Note(velocity=100, pitch=note_number, start=start, end=start + 0.5)
        instrument.notes.append(note_obj)
        start += 0.5

    midi.instruments.append(instrument)
    midi.write(output_file)

# Streamlit UI
st.title("LSTM Music Generator 🎶")
st.write("Generate music using a trained LSTM model and customize your inputs.")

# Load model and metadata
if "model" not in st.session_state:
    st.session_state.model = load_model()

if "vocab" not in st.session_state:
    st.session_state.vocab = vocab

if "int_to_note" not in st.session_state:
    st.session_state.int_to_note = {num: note for note, num in note_to_int.items()}

# User Inputs
st.sidebar.header("Input Settings")
num_notes = st.sidebar.slider("Number of Notes to Generate", min_value=50, max_value=500, value=100, step=10)
temperature = st.sidebar.slider("Creativity (Temperature)", min_value=0.1, max_value=2.0, value=1.0, step=0.1)

start_option = st.sidebar.selectbox(
    "Choose Starting Sequence",
    options=["Random", "Upload MIDI File"],
    index=0
)

if start_option == "Random":
    start_seq = input_sequences[np.random.randint(len(input_sequences))]
elif start_option == "Upload MIDI File":
    uploaded_file = st.sidebar.file_uploader("Upload a MIDI File", type=["mid", "midi"])
    if uploaded_file:
        midi_data = pretty_midi.PrettyMIDI(uploaded_file)
        start_seq = [note.pitch for instrument in midi_data.instruments for note in instrument.notes][:50]
    else:
        st.warning("Upload a MIDI file to proceed.")
        start_seq = None

# Generate Music Button
if st.button("Generate Music 🎹") and start_seq is not None:
    generated_notes = generate_music(
        st.session_state.model,
        st.session_state.int_to_note,
        start_seq,
        num_notes=num_notes,
        temperature=temperature
    )
    notes_to_midi(generated_notes, "output.mid")
    st.success("Music generated successfully! 🎵")
    st.audio("output.mid", format="audio/midi")
    st.download_button(
        label="Download MIDI File",
        data=open("output.mid", "rb").read(),
        file_name="generated_music.mid",
        mime="audio/midi"
    )

# Visualization (Optional)
if st.checkbox("Show Generated Notes"):
    st.write("Generated Notes:", generated_notes)
"""
with open("app.py", "w") as f:
    f.write(code)


In [6]:
!mkdir -p /content/project_folder
!mv app.py /content/project_folder/


In [7]:
%cd /content/project_folder/


/content/project_folder


In [8]:
!pip install streamlit -q

In [13]:
!wget -q -O - ipv4.icanhazip.com

35.185.134.180


In [ ]:
!npx localtunnel --port 8501 --subdomain your_subdomain --password your_password

⠙⠹⠸⠼⠴y
y


In [11]:
'''!ngrok authtoken 2rTvsPLocvOtK9f7hNOzkTECM3P_5Z6NEju4qHerjRAfzt4y7'''

'!ngrok authtoken 2rTvsPLocvOtK9f7hNOzkTECM3P_5Z6NEju4qHerjRAfzt4y7'

In [12]:
'''from pyngrok import ngrok

# Open a tunnel to the streamlit app
public_url = ngrok.connect(8501)
print('Streamlit app is accessible at:', public_url)'''


SyntaxError: incomplete input (<ipython-input-12-219889eedb59>, line 1)